In [10]:
import argparse
import torch.backends.cudnn as cudnn
import torchvision.utils as vutils
from pytorch3d.io import IO

#cudnn.benchmark = True

import utils
from utils.util_3d import init_mesh_renderer, sdf_to_mesh
from utils.qual_util import save_mesh_as_gif
from utils.demo_util import get_shape_comp_model, get_shape_comp_opt
from utils.qual_util import get_partial_shape_by_range
from preprocess.process_one_mesh import process_obj

%reload_ext autoreload
%autoreload 2

In [2]:
def get_parser():
    parser = argparse.ArgumentParser(description="MeshConv")
    parser.add_argument("-i", "--input", type=str, required=True)
    parser.add_argument("-o", "--output", type=str, default="output.obj")
    parser.add_argument("-model_path", type=str, default="saved_ckpt/rand_tf-snet_code-all-LR1e-4-clean-epoch200.pth")
    args = parser.parse_args()
    return args

def save_mesh(path, mesh):
    IO().save_mesh(mesh, path)

In [22]:
class AutoSDFComp():
    def __init__(self, mesh_path, model_path):
        self.load_pretrained_model(model_path)
        self.load_mesh(mesh_path)

    def load_pretrained_model(self, path):
        self.opt = get_shape_comp_opt(gpu_id=1)
        self.model = get_shape_comp_model(self.opt)
        self.model.load_ckpt(path)
        self.model.eval()

    def load_mesh(self, path):
        sdf_file = process_obj(path)
        min_x, max_x = -1., 0.
        min_y, max_y = -1., 0.
        min_z, max_z = -1., 0.
        sdf = utils.util_3d.read_sdf(sdf_file).to(self.opt.device)
        input_range = {'x1': min_x, 'x2': max_x, 'y1': min_y, 'y2': max_y, 'z1': min_z, 'z2': max_z}
        self.input = get_partial_shape_by_range(sdf, input_range)

    def completion(self):
        input_mesh, comp_sdf = self.model.shape_comp(self.input, bs=9, topk=30)
        save_mesh("input.obj", input_mesh)
        gen_mesh = sdf_to_mesh(comp_sdf)
        return gen_mesh

In [23]:
model_path = "saved_ckpt/rand_tf-snet_code-all-LR1e-4-clean-epoch200.pth"
mesh_path = "demo_data/chair_model.obj"
mesh_path = "demo_data/fandisk_original.obj"
compnet = AutoSDFComp(mesh_path, model_path)

[*] Enc has Attn at i_level, i_block: 3, 0
Working with z of shape (1, 256, 8, 8, 8) = 131072 dimensions.
[*] Dec has Attn at i_level, i_block: 3, 0
[*] VQVAE: weight successfully load from: saved_ckpt/pvqvae-snet-all-LR1e-4-T0.2-rerun-epoch140.pth
[*] Model has been created: Rand-Transformer-Model
[*] "rand_tf" initialized.
[*] weight successfully load from: saved_ckpt/rand_tf-snet_code-all-LR1e-4-clean-epoch200.pth
[*] weight successfully load from: saved_ckpt/rand_tf-snet_code-all-LR1e-4-clean-epoch200.pth
[*] creating tmp/for_sdf/sdf/fandisk_original/isosurf.sdf
[*] trimesh_load: demo_data/fandisk_original.obj
[*] export_mesh:  tmp/for_sdf/norm_mesh/fandisk_original/pc_norm.obj
[*] command: ./preprocess/isosurface/computeDistanceField tmp/for_sdf/norm_mesh/fandisk_original/pc_norm.obj 256 256 256 -s  -e 1.3 -o 0.dist -m 1 -c
Expansion ratio is: 1.3.
sigma = 0.0148704 = 2 grids
bandwidth = 0.037176 = 5 grids
Data output format is: IEEE single (32-bit) precision.
UseCubeBox: 1
******

In [24]:
out_mesh = compnet.completion()

[*] autoregressively inferencing...: 100%|██████████| 448/448 [00:34<00:00, 12.91it/s]


In [25]:
save_mesh("output.obj", out_mesh[0])